In [1]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/opt/ml/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
! sleep 30

In [2]:
! /opt/ml/elasticsearch-7.9.2/bin/elasticsearch-plugin install analysis-nori

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Failed installing analysis-nori
-> Rolling back analysis-nori
-> Rolled back analysis-nori
ERROR: plugin directory [/opt/ml/elasticsearch-7.9.2/plugins/analysis-nori] already exists; if you need to update the plugin, uninstall it first using command 'remove analysis-nori'


In [3]:
es_server.kill()

In [4]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/opt/ml/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
! sleep 30

In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')

In [6]:
es.info()

{'name': '3897a643c8f2',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'snNObfPqSYyWYnVkZ816XQ',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [7]:
es.indices.create(index = 'document',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "nori_number"]
                                  }
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              }
                          }
                      }
                  }
                  )

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [document/5qdKsxhsROKTI4P2bHCXiQ] already exists')

In [9]:
es.indices.get('document')

{'document': {'aliases': {},
  'mappings': {'properties': {'text': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'},
    'title': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'}}},
  'settings': {'index': {'number_of_shards': '1',
    'provided_name': 'document',
    'similarity': {'my_similarity': {'type': 'BM25'}},
    'creation_date': '1620968855773',
    'analysis': {'filter': {'my_shingle_f': {'type': 'shingle'}},
     'analyzer': {'my_analyzer': {'filter': ['lowercase',
        'my_shingle_f',
        'nori_readingform',
        'nori_number'],
       'decompound_mode': 'mixed',
       'type': 'custom',
       'stopwords': '_korean_',
       'tokenizer': 'nori_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': '5qdKsxhsROKTI4P2bHCXiQ',
    'version': {'created': '7090299'}}}}}

In [10]:
import json
import pandas as pd

with open('/opt/ml/code/data/wikipedia_documents.json', 'r') as f:
    wiki_data = pd.DataFrame(json.load(f)).transpose()

In [13]:
from tqdm.notebook import tqdm

for num in tqdm(range(len(wiki_data))):
    es.index(index='document', body = {"title" : wiki_data['title'][num], "text" : wiki_data['text'][num]})

In [14]:
question = input()

query = {
    'query':{
        'bool':{
            'must':[
                      {'match':{'text':question}}
            ],
            'should':[
                      {'match':{'text':question}}
            ]
        }
    }
}

doc = es.search(index='document',body=query,size=10)['hits']['hits']

doc

[]